In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [3]:
random_state = 0

# IMPORT DATASET

In [4]:
dataset_path = parentdir + "/datasets/PhalangesOutlinesCorrect/"

In [5]:
dataset_name = "phalanges"

In [6]:
X = pd.read_csv(dataset_path + "PhalangesOutlinesCorrect_TRAIN.txt", header=None, delim_whitespace=True)
y_all = np.array(X[0]).astype("int")
#y_all[y_all == -1] = 0

X_all = X.iloc[:,1:].values
X_all = X_all.reshape((X_all.shape[0], X_all.shape[1], 1))

In [7]:
print("X SHAPE: ", X_all.shape)
print("y SHAPE: ", y_all.shape)
unique, counts = np.unique(y_all, return_counts=True)
print("\nCLASSES BALANCE")
for i, label in enumerate(unique):
    print(label, ": ", round(counts[i]/sum(counts), 2))

X SHAPE:  (1800, 80, 1)
y SHAPE:  (1800,)

CLASSES BALANCE
0 :  0.35
1 :  0.65


In [8]:
X_test = pd.read_csv(dataset_path + "PhalangesOutlinesCorrect_TEST.txt", header=None, delim_whitespace=True)
y_test = np.array(X_test[0]).astype("int")
#y_all[y_all == -1] = 0

X_test = X_test.iloc[:,1:].values
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [9]:
print("X SHAPE: ", X_test.shape)
print("y SHAPE: ", y_test.shape)

X SHAPE:  (858, 80, 1)
y SHAPE:  (858,)


## DATASET SPLIT

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# BLACKBOX/EXPLANATION SETS SPLIT
X_train, X_exp, y_train, y_exp = train_test_split(X_all, y_all, 
                                                  test_size=0.3, stratify = y_all, random_state=random_state)

# BLACKBOX TRAIN/VALIDATION SETS SPLIT
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, stratify = y_train, random_state=random_state)

# EXPLANATION TRAIN/TEST SETS SPLIT
X_exp_train, X_exp_test, y_exp_train, y_exp_test = train_test_split(X_exp, y_exp, 
                                                                    test_size=0.2, 
                                                                    stratify = y_exp, 
                                                                    random_state=random_state)

# EXPLANATION TRAIN/VALIDATION SETS SPLIT
X_exp_train, X_exp_val, y_exp_train, y_exp_val = train_test_split(X_exp_train, y_exp_train, 
                                                                  test_size=0.2, 
                                                                  stratify = y_exp_train, 
                                                                  random_state=random_state)

print("SHAPES:")
print("BLACKBOX TRAINING SET: ", X_train.shape)
print("BLACKBOX VALIDATION SET: ", X_val.shape)
print("BLACKBOX TEST SET: ", X_test.shape)
print("EXPLANATION TRAINING SET: ", X_exp_train.shape)
print("EXPLANATION VALIDATION SET: ", X_exp_val.shape)
print("EXPLANATION TEST SET: ", X_exp_test.shape)

SHAPES:
BLACKBOX TRAINING SET:  (1008, 80, 1)
BLACKBOX VALIDATION SET:  (252, 80, 1)
BLACKBOX TEST SET:  (858, 80, 1)
EXPLANATION TRAINING SET:  (345, 80, 1)
EXPLANATION VALIDATION SET:  (87, 80, 1)
EXPLANATION TEST SET:  (108, 80, 1)


In [12]:
n_timesteps, n_outputs, n_features = X_train.shape[1], len(np.unique(y_all)), 1 
print("TIMESTEPS: ", n_timesteps)
print("N. LABELS: ", n_outputs)

TIMESTEPS:  80
N. LABELS:  2


# BLACKBOX MODEL

In [13]:
"""import importlib
importlib.reload(blackboxes)"""
import keras
from sklearn.metrics import accuracy_score, mean_squared_error
from joblib import dump, load
import time
from blackboxes import *
import blackboxes

Using TensorFlow backend.
/Users/francesco/anaconda3/envs/tesi/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/francesco/anaconda3/envs/tesi/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/francesco/anaconda3/envs/tesi/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/frances

## resnet

In [14]:
blackbox = build_resnet(n_timesteps, n_outputs)
blackbox.load_weights(parentdir + "/blackbox_checkpoints/phalanges_blackbox_resnet_20191101_164247_best_weights_+0.86_.hdf5")
resnet = blackbox

Instructions for updating:
Colocations handled automatically by placer.


## simple CNN

In [15]:
blackbox = build_simple_CNN(n_timesteps, n_outputs)
blackbox.load_weights(parentdir + "/blackbox_checkpoints/phalanges_blackbox_simpleCNN_20191101_170209_best_weights_+0.83_.hdf5")
simplecnn = blackbox

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## KNN

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, mean_squared_error, accuracy_score
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [17]:
knn = load(parentdir + "/blackbox_checkpoints/phalanges_blackbox_knn_20191101_171534.joblib")

# AUTOENCODER

In [18]:
"""import importlib
importlib.reload(autoencoders)"""
import autoencoders
from autoencoders import *

## Standard

In [19]:
params = {"input_shape": (n_timesteps,1),
          "n_blocks": 8, 
          "latent_dim": 40,
          "encoder_latent_layer_type": "simple",
          "encoder_args": {"filters":[2,4,8,16,32,64,128,256], 
                            "kernel_size":[21,18,15,13,11,8,5,3], 
                            "padding":"same", 
                            "activation":"elu", 
                            "pooling":[1,1,1,1,1,1,1,2]}
         }

aut = Autoencoder(verbose = False, **params)
encoder, decoder, autoencoder = aut.build()
autoencoder.load_weights(parentdir + "/autoencoder_checkpoints/phalanges_autoencoder_20191103_211535_best_weights_+0.0010_.hdf5")

# GLOBAL VS LOCAL SHAPELET TREE

In [20]:
# 50 test set
_, X_exp_test_50, _, y_exp_test_50 = train_test_split(X_exp_test, y_exp_test, 
                                                                  test_size=50, 
                                                                  stratify = y_exp_test, 
                                                                  random_state=random_state)

In [21]:
import agnosticlocalexplainer
from agnosticlocalexplainer import *

from myutils import BlackboxPredictWrapper
import time
from agnosticglobalexplainer import AgnosticGlobalExplainer, save_shapelet_model, load_shapelet_model
from joblib import dump
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, coverage_error
from global_vs_local_surrogate import build_agnostic_local_explainers, print_report, massive_save_agnostic_local_explainers, massive_load_agnostic_local_explainers, get_all_predictions

/Users/francesco/anaconda3/envs/tesi/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


## RESNET

In [22]:
autoencoder = autoencoder
encoder = autoencoder.layers[1]
decoder = autoencoder.layers[2]
blackbox = resnet
blackbox_input_dimensions = 3
labels = ["WALKING", "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS", "SITTING", "STANDING", "LAYING"]

In [ ]:
max_iter = 50
file_path = parentdir + "/agnostic_explainers/" + dataset_name + "_" + time.strftime("%Y%m%d_%H%M%S")

In [ ]:
import platform
if platform.system() == "Windows":
    os.makedirs(file_path + "/")
else: os.mkdir(file_path + "/")

In [37]:
%%time
global_surrogate = AgnosticGlobalExplainer(random_state = random_state, 
                                           max_iter = max_iter, 
                                           distance_quantile_threshold = np.array(list(range(1,10)))/10,
                                           labels = labels)
global_surrogate.fit(X_exp_train[:,:,0], blackbox_predict.predict(X_exp_train))

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    2.8s finished


In [22]:
%%time
agnostic_explainers = build_agnostic_local_explainers(blackbox, 
                               encoder, 
                               decoder, 
                               autoencoder, 
                               X_exp_test_50, 
                               y_exp_test_50,
                               blackbox_input_dimensions = blackbox_input_dimensions,
                               labels = labels,
                               size = 1000,
                               neigh_type = "geneticp",
                               ngen = 10,
                              max_iter=max_iter,
                              random_state = random_state,
                              distance_quantile_threshold = np.array(list(range(1,10)))/10
                               )


Neighborhood Generation
calculating feature values
gen	nevals	avg     	min     	max     
0  	500   	0.496654	0.496654	0.496654
1  	425   	0.526635	0.496654	0.993097
2  	434   	0.569137	0.496654	0.993249
3  	441   	0.604086	0.500285	0.993249
4  	422   	0.634237	0.500335	0.993274
5  	412   	0.663724	0.49984 	0.993274
6  	418   	0.685824	0.500308	0.993251
7  	418   	0.720994	0.500323	0.993248
8  	432   	0.732127	0.500485	0.993248
9  	412   	0.731842	0.500308	0.993248
10 	417   	0.734599	0.500296	0.993248
gen	nevals	avg	min	max
0  	500   	0.5	0.5	0.5
1  	423   	0.71907	0.499405	0.991901
2  	436   	0.942117	0.499186	0.991937
3  	419   	0.982799	0.499542	0.991937
4  	422   	0.979182	0.499274	0.991937
5  	417   	0.977398	0.499351	0.991937
6  	410   	0.977384	0.499421	0.991937
7  	426   	0.977728	0.499271	0.991937
8  	410   	0.981799	0.499518	0.991937
9  	433   	0.981702	0.499525	0.991937
10 	428   	0.976748	0.499573	0.991937
synthetic neighborhood class counts {0: 915, 1: 285}
Fitting 3 fold

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    6.2s finished


1 / 50

Neighborhood Generation
calculating feature values
gen	nevals	avg     	min     	max     
0  	500   	0.496654	0.496654	0.496654
1  	425   	0.518616	0.496654	0.99305 
2  	434   	0.55508 	0.496654	0.993307
3  	441   	0.587018	0.49763 	0.993307
4  	422   	0.611382	0.498062	0.993307
5  	412   	0.644722	0.497587	0.993191
6  	418   	0.657623	0.49813 	0.993235
7  	418   	0.686009	0.497844	0.993235
8  	432   	0.699791	0.497156	0.993191
9  	412   	0.73117 	0.498053	0.993191
10 	417   	0.716169	0.497698	0.993253
gen	nevals	avg	min	max
0  	500   	0.5	0.5	0.5
1  	423   	0.7196	0.499068	0.993149
2  	436   	0.936631	0.499016	0.993149
3  	419   	0.978573	0.498843	0.993149
4  	422   	0.978553	0.498726	0.993149
5  	417   	0.986125	0.504259	0.993139
6  	410   	0.976309	0.498898	0.993143
7  	426   	0.974742	0.49891 	0.99315 
8  	410   	0.98062 	0.499295	0.993143
9  	433   	0.982084	0.499026	0.993143
10 	428   	0.983927	0.499504	0.993124
synthetic neighborhood class counts {0: 937, 1: 263}
Fitting 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 273 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    4.5s finished


2 / 50

Neighborhood Generation
calculating feature values
gen	nevals	avg     	min     	max     
0  	500   	0.496654	0.496654	0.496654
1  	425   	0.733415	0.496654	0.993193
2  	434   	0.96006 	0.496654	0.993264
3  	441   	0.984261	0.49978 	0.993272
4  	422   	0.984104	0.499683	0.993272
5  	412   	0.986302	0.499511	0.993269
6  	418   	0.987049	0.499487	0.993279
7  	418   	0.988381	0.499984	0.993279
8  	432   	0.988105	0.499907	0.993279
9  	412   	0.990341	0.539191	0.993279
10 	417   	0.982905	0.499699	0.993307
gen	nevals	avg	min	max
0  	500   	0.5	0.5	0.5
1  	423   	0.506194	0.498849	0.99051
2  	436   	0.512751	0.498646	0.992627
3  	419   	0.527605	0.498611	0.992627
4  	422   	0.541831	0.498803	0.992627
5  	417   	0.575383	0.498695	0.99291 
6  	410   	0.610622	0.498444	0.992916
7  	426   	0.624177	0.498474	0.99291 
8  	410   	0.641474	0.498572	0.99291 
9  	433   	0.646872	0.498376	0.99291 
10 	428   	0.655006	0.498482	0.992995
synthetic neighborhood class counts {0: 863, 1: 265}
Fitting

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    3.2s finished


49 / 50

Neighborhood Generation
calculating feature values
gen	nevals	avg     	min     	max     
0  	500   	0.496654	0.496654	0.496654
1  	425   	0.542885	0.496654	0.993256
2  	434   	0.577399	0.496654	0.993306
3  	441   	0.618834	0.499318	0.993306
4  	422   	0.658361	0.49909 	0.993264
5  	412   	0.695961	0.499214	0.993264
6  	418   	0.706595	0.499478	0.993264
7  	418   	0.722975	0.499409	0.993256
8  	432   	0.726238	0.498738	0.993257
9  	412   	0.742793	0.499469	0.993257
10 	417   	0.746445	0.499201	0.993257
gen	nevals	avg	min	max
0  	500   	0.5	0.5	0.5
1  	423   	0.704948	0.499279	0.992531
2  	436   	0.930526	0.499262	0.992531
3  	419   	0.973303	0.499126	0.992531
4  	422   	0.970734	0.49888 	0.992556
5  	417   	0.977216	0.499461	0.992556
6  	410   	0.980304	0.498818	0.992556
7  	426   	0.973892	0.499354	0.992594
8  	410   	0.977949	0.499433	0.992567
9  	433   	0.980943	0.499066	0.992567
10 	428   	0.974992	0.499604	0.992559
synthetic neighborhood class counts {0: 898, 1: 302}
Fitti

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 175 tasks      | elapsed:    2.4s


50 / 50
CPU times: user 5h 57min 21s, sys: 18min 8s, total: 6h 15min 30s
Wall time: 5h 31min 50s


[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    3.1s finished


In [38]:
results_df = get_all_predictions(agnostic_explainers, global_surrogate, blackbox_predict, X_exp_test_50, blackbox_predict.predict(X_exp_train))
results_df.to_csv(file_path + "/" + "results_df.csv", sep = ";", index = False)
print_report(results_df)

local fidelity:  0.84
global fidelity:  0.56
reconstruction fidelity:  0.98


local coverage:  1.14
global coverage:  1.44


local classification report:
              precision    recall  f1-score   support

           0       0.70      1.00      0.83        19
           1       1.00      0.74      0.85        31

    accuracy                           0.84        50
   macro avg       0.85      0.87      0.84        50
weighted avg       0.89      0.84      0.84        50



global classification report:
              precision    recall  f1-score   support

           0       0.41      0.37      0.39        19
           1       0.64      0.68      0.66        31

    accuracy                           0.56        50
   macro avg       0.52      0.52      0.52        50
weighted avg       0.55      0.56      0.55        50



reconstrution classification report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97        19
           1

In [23]:
### SHAP

from myutils import stabilities_df
from shap_utils import shap_stability, shap_multi_stability

save_path = parentdir + "/stabilities/" + dataset_name + "_stability" + "_shap_" + time.strftime("%Y%m%d_%H%M%S")

params = {"background": "linear"}


#### RESNET
filename = save_path + "_resnet"
stabilities = shap_multi_stability(X_exp_test_50, resnet, 3, point_by_point = False, **params)
np.save(filename, stabilities)
stabilities_df(stabilities, len(X_exp_test_50)).to_csv(filename + ".csv", sep = ";", index = True)


#### CNN
filename = save_path + "_simplecnn"
stabilities = shap_multi_stability(X_exp_test_50, simplecnn, 3, point_by_point = False, **params)
np.save(filename, stabilities)
stabilities_df(stabilities, len(X_exp_test_50)).to_csv(filename + ".csv", sep = ";", index = True)


#### KNN
filename = save_path + "_knn"
stabilities = shap_multi_stability(X_exp_test_50, knn, 2, point_by_point = False, **params)
np.save(filename, stabilities)
stabilities_df(stabilities, len(X_exp_test_50)).to_csv(filename + ".csv", sep = ";", index = True)

In [23]:
%%time
from global_vs_local_surrogate import calculate_shapelet_stability
### shapelets

n_neighbors = 30

#### KNN
file_path = parentdir + "/agnostic_explainers/phalanges_geneticp_knn_20200112_230219/"
agnostic_explainers = massive_load_agnostic_local_explainers(file_path, verbose = True)

save_path = parentdir + "/stabilities/" + dataset_name + "_stability" + "_shapelets_" + time.strftime("%Y%m%d_%H%M%S")
filename = save_path + "_knn"

stabilities = calculate_shapelet_stability(agnostic_explainers, X_exp_test_50, n_neighbors = n_neighbors)
np.save(filename, stabilities)


#### RESNET
file_path = parentdir + "/agnostic_explainers/phalanges_geneticp_resnet_20191223_160501/"
agnostic_explainers = massive_load_agnostic_local_explainers(file_path, verbose = True)

save_path = parentdir + "/stabilities/" + dataset_name + "_stability" + "_shapelets_" + time.strftime("%Y%m%d_%H%M%S")
filename = save_path + "_resnet"
stabilities = calculate_shapelet_stability(agnostic_explainers, X_exp_test_50, n_neighbors = n_neighbors)
np.save(filename, stabilities)


#### CNN
file_path = parentdir + "/agnostic_explainers/phalanges_geneticp_simplecnn_20200109_181310/"
agnostic_explainers = massive_load_agnostic_local_explainers(file_path, verbose = True)

save_path = parentdir + "/stabilities/" + dataset_name + "_stability" + "_shapelets_" + time.strftime("%Y%m%d_%H%M%S")
filename = save_path + "_simplecnn"
stabilities = calculate_shapelet_stability(agnostic_explainers, X_exp_test_50, n_neighbors = n_neighbors)
np.save(filename, stabilities)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
1 / 50
2 / 50
3 / 50
4 / 50
5 / 50
6 / 50
7 / 50
8 / 50
9 / 50
10 / 50
11 / 50
12 / 50
13 / 50
14 / 50
15 / 50
16 / 50
17 / 50
18 / 50
19 / 50
20 / 50
21 / 50
22 / 50
23 / 50
24 / 50
25 / 50
26 / 50
27 / 50
28 / 50
29 / 50
30 / 50
31 / 50
32 / 50
33 / 50
34 / 50
35 / 50
36 / 50
37 / 50
38 / 50
39 / 50
40 / 50
41 / 50
42 / 50
43 / 50
44 / 50
45 / 50
46 / 50
47 / 50
48 / 50
49 / 50
50 / 50
1 / 50
2 / 50
3 / 50
4 / 50
5 / 50
6 / 50
7 / 50
8 / 50
9 / 50
10 / 50
11 / 50
12 / 50
13 / 50
14 / 50
15 / 50
16 / 50
17 / 50
18 / 50
19 / 50
20 / 50
21 / 50
22 / 50
23 / 50
24 / 50
25 / 50
26 / 50
27 / 50
28 / 50
29 / 50
30 / 50
31 / 50
32 / 50
33 / 50
34 / 50
35 / 50
36 / 50
37 / 50
38 / 50
39 / 50
40 / 50
41 / 50
42 / 50
43 / 50
44 / 50
45 / 50
46 / 50
47 / 50
48 / 50
49 / 50
50 / 50
1 / 50
2 / 50
3 / 50
4 / 50
5 / 50
6 / 50
7 / 50
8 / 50
9 / 50
10 / 50
11 / 50
12 / 50
13 / 